# calculate wind shear excedence map

In [ ]:
import iris

In [ ]:
direction = iris.load_cube('wind_direction.nc')
direction

In [ ]:
speed = iris.load_cube('wind_speed.nc')
speed

In [ ]:
from opscentretools import sheartools
shear = sheartools.calculate_shear(speed, direction)
shear

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

In [ ]:
client

In [ ]:
!export JUPYTERHUB_USER=jovyan

In [ ]:
!printenv